In [1]:
!pip install requests
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd 
import requests
import re
import os  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_KDM/CSEE5590-KDM/kdm_project/nsf_funding1.csv')
df['Solicitation URL'][1]
#%config InlineBackend.figure_format = 'retina'

'https://www.nsf.gov/publications/pub_summ.jsp?WT.z_pims_id=505684&ods_key=nsf23511'

In [10]:
solstr = []
f = open("nsf_sol.txt", "a")
#df['Solicitation URL'] = df['Solicitation URL'].str.replace('NaN', "")
df['Solicitation URL'].fillna('', inplace=True)   #dropping na
for row in df['Solicitation URL']:
    if 'https' in row:
        r = requests.get(row)
        #----------------- getting actual URL ------------------
        soup = bs(r.content, 'html.parser')
        s = soup.find('div', id='inner-content')
        #print(s)
        nsf = s.find('a', string="HTML")
        url = nsf.get('href')     ## https://www.nsf.gov/pubs/2023/nsf23512/nsf23512.htm
        #print(url)
        #------------------------ END -----------------------#

        # ------------- Scrapping required tags -------------#
        p_list = []
        if 'https' in url:
            nsf_url = requests.get(url)
            #print("==================================================",url)
            soup = bs(nsf_url.content, 'html.parser')
            #print("------------------- NSF URL:", soup.getText)
            solicitation = soup.find('div', class_='solicitation')
            #print("sol ----------------------",solicitation)
            #print("------------------- find here:", row)
            #print("------------------- NSF URL:", url)
            #print("------------------- NSF URL:", solicitation)
            if solicitation:
                p = solicitation.find('tbody')
            else:
                p = soup.find('tbody')
            #print("td ----------------------",p)
            td = p.find('td')
            
            p = td.findAllNext(['h3','h4','p','li','em'])
            for sol in p:
                if 'ABOUT THE NATIONAL SCIENCE FOUNDATION' in str(sol.prettify()):
                    print(sol)
                    break
                text = sol.get_text("|", strip=True)
                p_list.append(text)
                #p_list.append('\n')
            text = str(p_list)
            text = text.replace(r"\r\n", r" ")
            text = text.replace("', '",' ')
            text = text.replace("  ",'')
            text = text.replace("National Science Foundation",'')
            solstr.append(text)
            #print(solstr)
            #f.write(text+ "\n")
        else:
            nsf_url = requests.get(row)
            solstr.append(url)
            #print("not ok==================================================",nsf_url)
    else:
        solstr.append(row)
        text = str(solstr)
    #solstr.append(data)
df['solsumm'] = solstr
#f.close()

Streaming output truncated to the last 5000 lines.
<p>Approximately $18-20 million is expected to be available for new awards in FY 2021. The size and duration of any individual request should be justified by the amount and complexity of the work to be accomplished. As a rule, the larger the budget, the greater the expected impact on the biological research community.</p>
<h4>
            Eligibility Information
          </h4>
<p><strong>Who May Submit Proposals:</strong></p>
<blockquote>
<p>Proposals may only be submitted by the following:</p>
<ul>
<li> Institutions of Higher Education (IHEs) - Two- and four-year IHEs (including community colleges) accredited in, and having a campus located in the US, acting
                on behalf of their faculty members. Special Instructions for International Branch Campuses of US IHEs: If the proposal includes funding to be provided to an
                international branch campus of a US institution of higher education (including through use 

In [13]:
os.makedirs('biin', exist_ok=True) 
df.to_csv('biin/biin_sol.csv', encoding='utf-8', index=False)